In [ ]:
# scraping Q&As of Li Fei from Haodf.com

import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re
import random
import datetime

In [2]:
# weburl = "http://www.haodf.com/wenda/dflifei_g_4642049195.htm" # a test page
webheader = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) \
        AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36'}
urls = []
data = []

def getSinglePage(pageurl):
    req = urllib.request.Request(url=pageurl, headers=webheader)
    try:
        webPage = urllib.request.urlopen(req)
        """
        print(type(webPage))
        print(webPage.geturl())
        print(webPage.info())
        print(webPage.getcode())
        """
    except:
        print('\033[33m Warning:\033[0m Fail to open:', pageurl)
        pass # failed +1
        return None
    else:
        bsObj = BeautifulSoup(webPage, "lxml")
        # questions = bsObj.findAll("div", {"class":"h_s_cons_info"})
        # answers = bsObj.findAll("div", {"class":"h_s_cons_docs"})
        patient = bsObj.find(lambda tag: tag.get_text() == '病情描述：')
        if patient:
            symptom = ''.join(list(map(str, patient.next_siblings))).replace('<br/>', '')
            # get rid of bingli attachment
            symptom = re.sub(r'<[^>]*>[^<>]*</[^>]*>|\n|\r', '', symptom)
            patient = patient.parent.parent.parent.parent.parent # any better solution?
            symptom_time = patient.find("div", {"class":"yh_l_times"}).get_text()
            for reply in patient.next_siblings: # diagnosis follows symptom
                try:
                    diagnosis = reply.find("div", {"class":"h_s_cons_docs"})
                    diagnosis = diagnosis.find("h3").get_text()
                except:
                    continue # not a reply by doctor, keep searching...
                else: # valid Q&A completes here
                    diagnosis_time = reply.find("div", {"class":"yh_l_times"}).get_text()
                    """
                    print(symptom)
                    print(symptom_time)
                    print(diagnosis)
                    print(diagnosis_time)
                    """
                    data.append({'url':pageurl, \
                                 'symptom':symptom, 'symptom_time':symptom_time, \
                                 'diagnosis':diagnosis, 'diagnosis_time':diagnosis_time})
                    break # only the first reply is valid
        else: # not Q&As of desired pattern (see README.md)
            pass # dirty data +1

# getSinglePage(weburl)

for i in range(1,97): # 1-96 pages, each has 25 posts
    listurl = ''.join(['http://dflifei.haodf.com/zixun/list.htm?type=&p=',str(i)])
    req = urllib.request.Request(url=listurl, headers=webheader)
    listPage = urllib.request.urlopen(req)
    print('Now scraping page', i)
    bsObj = BeautifulSoup(listPage, "lxml")
    listtable = bsObj.find("div", {"class":"zixun_list"})
    for post in listtable.findAll("a", {"class":"td_link"}):
        urls.append(post['href'])
        print('    Now scraping post', post['href'])
        getSinglePage(post['href'])
        print(len(data), '/', len(urls)) # timely feedback makes me happy

print('Scraped', len(data), 'valid data out of', len(urls), 'posts.')
frame = pd.DataFrame(data)
frame.to_pickle('../data/data-' \
                + datetime.datetime.today().strftime('%Y%m%d-%H.%M.%S') \
                + '.pickle')

Now scraping page 1
    Now scraping post http://www.haodf.com/wenda/dflifei_g_4666329975.htm
1 / 1
    Now scraping post http://www.haodf.com/wenda/dflifei_g_4301838002.htm
2 / 2
    Now scraping post http://www.haodf.com/wenda/dflifei_g_4551785617.htm
2 / 3
    Now scraping post http://www.haodf.com/wenda/dflifei_g_4409750380.htm
2 / 4
    Now scraping post http://www.haodf.com/wenda/dflifei_g_4666331357.htm
3 / 5
    Now scraping post http://www.haodf.com/wenda/dflifei_g_4648398706.htm
3 / 6
    Now scraping post http://www.haodf.com/wenda/dflifei_g_4642049195.htm
4 / 7
    Now scraping post http://www.haodf.com/wenda/dflifei_g_4666196792.htm
4 / 8
    Now scraping post http://www.haodf.com/wenda/dflifei_g_4661421053.htm
5 / 9
    Now scraping post http://www.haodf.com/wenda/dflifei_g_4665421732.htm
5 / 10
    Now scraping post http://www.haodf.com/wenda/dflifei_g_4665937456.htm
6 / 11
    Now scraping post http://www.haodf.com/wenda/dflifei_g_4664246450.htm
6 / 12
    Now scraping 